In [1]:
import numpy as np

def obj_to_ply_with_color_and_transform(input_obj_path, output_ply_path, center_object=True, inverse_y=False, default_color=(0.5, 0.5, 0.5)):
    """
    Lit un fichier OBJ, assigne une couleur par défaut aux sommets sans couleur,
    applique des transformations (centrage, inversion Y) et sauvegarde en format PLY.

    Args:
        input_obj_path (str): Chemin du fichier OBJ d'entrée.
        output_ply_path (str): Chemin du fichier PLY de sortie.
        center_object (bool): Si True, l'objet sera centré.
        inverse_y (bool): Si True, l'objet sera inversé selon l'axe Y.
        default_color (tuple): Couleur RGB (0.0-1.0) à assigner aux sommets sans couleur.
                               Par défaut, c'est un gris moyen (0.5, 0.5, 0.5).
    """
    vertices = []
    vertex_colors = []
    faces = []
    normals = [] # Les normales ne sont pas directement écrites dans le PLY ici, mais gardées si besoin pour d'autres usages.
    texcoords = [] # Les coordonnées de texture ne sont pas directement écrites dans le PLY ici.

    print(f"Lecture du fichier OBJ : {input_obj_path}")
    with open(input_obj_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if not parts:
                continue

            if parts[0] == 'v':
                v = [float(p) for p in parts[1:4]]
                vertices.append(v)
                if len(parts) == 7:  # Si des couleurs sont présentes (x y z r g b)
                    c = [float(p) for p in parts[4:7]]
                    # Convertir les couleurs de 0.0-1.0 à 0-255 pour le PLY
                    vertex_colors.append([int(val * 255) for val in c])
                else:
                    # Assignation de la couleur par défaut (gris)
                    vertex_colors.append([int(val * 255) for val in default_color])
            elif parts[0] == 'f':
                # Les faces dans OBJ peuvent être 'v', 'v/vt', 'v/vt/vn'. Nous prenons juste l'indice du sommet.
                current_face = []
                for part in parts[1:]:
                    # Split par '/' et prendre le premier élément (l'indice du sommet)
                    current_face.append(int(part.split('/')[0]) - 1) # -1 car OBJ est 1-indexé, Python 0-indexé
                faces.append(current_face)
            # Nous ignorons 'vn' et 'vt' pour l'export PLY simple qui se concentre sur v et f,
            # mais vous pouvez les ajouter à l'en-tête PLY si nécessaire.

    if not vertices:
        print("Aucun sommet trouvé dans le fichier OBJ. Opération annulée.")
        return

    vertices_np = np.array(vertices)

    # --- Centrage de l'objet ---
    if center_object:
        min_coords = np.min(vertices_np, axis=0)
        max_coords = np.max(vertices_np, axis=0)
        center = (min_coords + max_coords) / 2.0
        print(f"Centrage de l'objet par rapport à : {center}")
        vertices_np = vertices_np - center

    # --- Inversion selon l'axe Y ---
    if inverse_y:
        vertices_np[:, 1] *= -1
        print("Inversion des sommets selon l'axe Y appliquée.")

    print(f"Sauvegarde de l'objet transformé en PLY dans : {output_ply_path}")

    # --- Écriture du fichier PLY ---
    with open(output_ply_path, 'w') as f:
        # En-tête du fichier PLY
        f.write("ply\n")
        f.write("format ascii 1.0\n")
        f.write(f"element vertex {len(vertices_np)}\n")
        f.write("property float x\n")
        f.write("property float y\n")
        f.write("property float z\n")
        f.write("property uchar red\n") # uchar pour les couleurs de 0-255
        f.write("property uchar green\n")
        f.write("property uchar blue\n")
        f.write(f"element face {len(faces)}\n")
        f.write("property list uchar int vertex_indices\n") # Pour les faces (polygones)
        f.write("end_header\n")

        # Écriture des données des sommets
        for i, v_transformed in enumerate(vertices_np):
            color = vertex_colors[i]
            f.write(f"{v_transformed[0]:.6f} {v_transformed[1]:.6f} {v_transformed[2]:.6f} ")
            f.write(f"{color[0]} {color[1]} {color[2]}\n")

        # Écriture des données des faces
        for face in faces:
            # Le format PLY pour les faces est : <nombre de sommets> <indice1> <indice2> ...
            f.write(f"{len(face)} " + " ".join(map(str, face)) + "\n")

    print("Traitement terminé avec succès.")

In [ ]:
input_file = "../graphics/visapp/3d/gorgoile/gorgoile_segmented_color.obj"
#output_file = input_file.replace('_pose_i7-j2.obj', '_centered.ply')
output_file = input_file.replace('gorgoile_segmented_color.obj', 'eyes.ply')

print("\n--- Transformation de l'OBJ en PLY avec centrage, inversion Y et couleurs par défaut ---")
obj_to_ply_with_color_and_transform(input_file, output_file,
                                    center_object=True,
                                    inverse_y=False,
                                    default_color=(0.7, 0.7, 0.7)) # Un gris un peu plus clair

print("Transformation terminée.")


--- Transformation de l'OBJ en PLY avec centrage, inversion Y et couleurs par défaut ---
Lecture du fichier OBJ : ../graphics/visapp/3d/gorgoile/Limper_gorgoile_jet.obj
Centrage de l'objet par rapport à : [  1.3636    -58.132361    0.2668495]
Sauvegarde de l'objet transformé en PLY dans : ../graphics/visapp/3d/gorgoile/Limper_gorgoile_jet_centered.ply
Traitement terminé avec succès.
Transformation terminée.
